In [1]:
import pandas as pd

In [2]:
import re

Задание 1
Напишите функцию, которая классифицирует фильмы из материалов занятия по правилам:

оценка 2 и ниже — низкий рейтинг;
оценка 4 и ниже — средний рейтинг;
оценка 4.5 и 5 — высокий рейтинг.
Результат классификации запишите в столбец class.

In [3]:
def classification(row):

    if row <= 2.0: class_column = "низкий рейтинг"
    elif row <= 4.0: class_column = "средний рейтинг"
    elif row == 4.5 or row == 5.0: class_column = "высокий рейтинг"
    return  class_column   

In [4]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
ratings['class'] = ratings['rating'].apply(classification)
joined = movies.merge(ratings, on='movieId', how='left')
joined[['title','class','rating']].head()

,title,class,rating
0,Toy Story (1995),средний рейтинг,3.0
1,Toy Story (1995),средний рейтинг,4.0
2,Toy Story (1995),высокий рейтинг,5.0
3,Toy Story (1995),низкий рейтинг,2.0
4,Toy Story (1995),средний рейтинг,3.0


Задание 2
Используйте файл keywords.csv.

Нужно написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определённому региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Результат классификации запишите в отдельный столбец region.

In [296]:
geo_data = {

'Центр': ['москв', 'тул', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

In [297]:
#список гласных букв, которые могут попадаться в конце названий городов
vowels = ["а", "о", "у", "ы","э", "е", "ё", "и", "ю", "я"]

In [298]:
#удаляем окончания у городов в словаре geo_data, чтобы не потерять слова с другими окончаниями в поисковых запросах
def transformation_dict(geo_data=geo_data, vowels=vowels):

    for key, value in geo_data.items():
        for region in value:             
            result = re.findall(r'\w\b', region)        
            if result[0] in vowels:
                result = re.sub(r'\w\b','', region)            
                geo_data[key][value.index(region)] =  result            
    return geo_data

In [299]:
keywords = pd.read_csv('keywords.csv')

In [300]:
def region(row, some_dict):

    for key, value in some_dict.items():             
        for region in value:                     
            if region in row:
                return  key
    return "undefined"   

In [301]:
keywords['region'] = keywords['keyword'].apply(region, some_dict = transformation_dict())

In [302]:
keywords.head()

,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined


In [303]:
keywords[keywords['region'] == 'Северо-Запад'].head()

,keyword,shows,region
370,авито ру санкт петербург,425134,Северо-Запад
564,погода в санкт петербурге,292489,Северо-Запад
1236,банк санкт петербург,174375,Северо-Запад
1346,санкт петербург,144232,Северо-Запад
2121,электронный дневник санкт петербургского образ...,94835,Северо-Запад


In [304]:
keywords[keywords['region'] == 'Центр'].head()

,keyword,shows,region
127,авито москва,979292,Центр
143,эхо москвы,889657,Центр
197,погода в москве,745745,Центр
414,погода в москве на 14 дней,400914,Центр
512,карта метро москвы,331857,Центр


In [305]:
keywords[keywords['region'] == 'Дальний Восток'].head()

,keyword,shows,region
1063,фарпост владивосток,176951,Дальний Восток
2175,дром хабаровск,98361,Дальний Восток
3206,дром владивосток,73084,Дальний Восток
3275,100 сп владивосток,76418,Дальний Восток
3687,авито хабаровск,63736,Дальний Восток


Задание 3 (бонусное)

Есть мнение, что раньше снимали настоящее кино, не то что сейчас. Ваша задача — проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия: файл movies.csv и ratings.csv из базы. Нужно проверить, верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

Вы не будете затрагивать субьективные факторы выставления этих рейтингов, а пройдётесь по алгоритму:

В переменную years запишите список из всех годов с 1950 по 2010 года.

Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте алгоритм:

для каждой строки пройдите по всем годам списка years;
если номер года присутствует в названии фильма, то функция возвращает этот год, как год выпуска;
если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год.
Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’.

Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга.

In [306]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [307]:
year = [i for i in range(1950,2011)]

In [308]:
def production_year(row, year):

    test = (re.sub(r'[(](\d{4})[)]', r'\1', row)).split(' ')        
    for item in year:        
        if str(item) in test:
            return str(item)
    return "1900"   

In [309]:
movies['year'] = movies['title'].apply(production_year, year=year)

In [310]:
joined = movies.merge(ratings, on='movieId', how='left')
joined.groupby('year').mean()['rating'].sort_values(ascending=False).head(15)

year
1957    4.014241
1972    4.011136
1952    4.000000
1954    3.994220
1951    3.983539
1974    3.978704
1962    3.952446
1950    3.915254
1977    3.905786
1964    3.841492
1959    3.841033
1958    3.823171
1975    3.821629
1969    3.817376
1967    3.814516
Name: rating, dtype: float64